In [1]:
from lbmpy.session import *
from lbmpy.phasefield.analytical import *


def laplacian(f):
    return Diff(Diff(f))

# Comparison of 3-phase model equations with paper

[Semprebon, Krüger, Kusumaatmaja] 2016 

#### Free Energy Function F
First we compare the transformed free energy function (23):

In [2]:
order_parameters = sp.symbols("rho phi psi")
κ = sp.symbols("kappa_:3")
α = sp.symbols("alpha")
ρ, φ, ψ = order_parameters
F, _ = free_energy_functional_3_phases(order_parameters, kappa=κ, interface_width=α)

eq_23 = κ[0] / 32 * (ρ + φ - ψ) ** 2 * (2 + ψ - ρ - φ) ** 2 \
        + α ** 2 * κ[0] / 8 * (Diff(ρ) + Diff(φ) - Diff(ψ)) ** 2 \
        + κ[1] / 32 * (ρ - φ - ψ) ** 2 * (2 + ψ - ρ + φ) ** 2 \
        + α ** 2 * κ[1] / 8 * (Diff(ρ) - Diff(φ) - Diff(ψ)) ** 2 \
        + κ[2] / 2 * ψ ** 2 * (1 - ψ) ** 2 + α ** 2 * κ[2] / 2 * (Diff(ψ)) ** 2

assert F - eq_23.expand() == 0

#### Chemical Potentials

This compares formulas (38), (39) and (40) from the paper, as typed in the next cell, to the automatically derived version which starts from the potential formulation above. 

For equation (38) the automatically derived version is different than the version reported in the paper.
See the code in the next cell for our correction.

In [3]:
def chemical_potential_formulas_from_paper(order_parameters=sp.symbols("rho phi psi"),
                                           κ=sp.symbols("kappa_:3"),
                                           α=sp.symbols("alpha")):
    ρ, φ, ψ = order_parameters

    # --------------------- μ_ρ formulas ---------------------------------------------

    # possibly wrong version in paper?
    μ_ρ_paper = κ[0] / 8 * (ρ + φ - ψ) * (ρ + φ - ψ - 2) * (ρ + φ - ψ - 1) \
                - κ[1] / 8 * (ρ - φ - ψ) * (ρ - φ - ψ - 2) * (ρ - φ - ψ - 1) \
                + α ** 2 / 4 * ((κ[0] + κ[1]) * (laplacian(ψ) - laplacian(φ)) +
                                (κ[1] - κ[0]) * laplacian(ρ))
    # corrections from paper: sign of kappa[1] term + different interface term
    μ_ρ = κ[0] / 8 * (ρ + φ - ψ) * (ρ + φ - ψ - 2) * (ρ + φ - ψ - 1) \
          + κ[1] / 8 * (ρ - φ - ψ) * (ρ - φ - ψ - 2) * (ρ - φ - ψ - 1) \
          - α ** 2 / 4 * ((κ[0] + κ[1]) * (laplacian(ρ) - laplacian(ψ)) +
                          (κ[0] - κ[1]) * laplacian(φ))

    # --------------------- μ_φ formulas ---------------------------------------------
    μ_φ = κ[0] / 8 * (ρ + φ - ψ) * (ρ + φ - ψ - 2) * (ρ + φ - ψ - 1) \
          - κ[1] / 8 * (ρ - φ - ψ) * (ρ - φ - ψ - 2) * (ρ - φ - ψ - 1) \
          + α ** 2 / 4 * ((κ[1] - κ[0]) * (laplacian(ρ) - laplacian(ψ)) -
                          (κ[0] + κ[1]) * laplacian(φ))

    # --------------------- μ_ψ formulas ---------------------------------------------

    μ_ψ = -κ[0] / 8 * (ρ + φ - ψ) * (ρ + φ - ψ - 2) * (ρ + φ - ψ - 1) \
          - κ[1] / 8 * (ρ - φ - ψ) * (ρ - φ - ψ - 2) * (ρ - φ - ψ - 1) \
          + κ[2] * ψ * (ψ - 1) * (2 * ψ - 1) + α ** 2 / 4 * ((κ[0] + κ[1]) * laplacian(ρ)
                                                             - (κ[1] - κ[0]) * laplacian(φ) - (
                                                                     κ[1] + κ[0] + 4 * κ[2]) * laplacian(ψ))

    return μ_ρ, μ_φ, μ_ψ


In [4]:
F, _ = free_energy_functional_3_phases(order_parameters)
μ_derived = chemical_potentials_from_free_energy(F, order_parameters)
μ_paper = chemical_potential_formulas_from_paper()


In [5]:
assert (μ_derived[0] - μ_paper[0]).expand() == 0
assert (μ_derived[1] - μ_paper[1]).expand() == 0
assert (μ_derived[2] - μ_paper[2]).expand() == 0